## Data

For this notebook, we will use the Reuters newswire dataset. This dataset consists of 11,228 newswires from the Reuters news agency. Each wire is encoded as a sequence of word indexes. Moreover, each wire is categorised into one of 46 topics, which will serve as our label. This dataset is available through the Keras API.

## Goal

We want to create a Multi-layer perceptron (MLP) & lstm (RNN) using Keras which we can train to classify news items into the specified 46 topics.


In [31]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical

seed = 1337
np.random.seed(seed)

## Import Dataset

In [32]:
from keras.datasets import reuters

max_words = 1000
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2,
                                                         seed=seed)
num_classes = np.max(y_train) + 1  # 46 topics

## 1) Train with Multilayer Perceptron Neural Network

## Preprocessing

In [33]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

## Label encoding

Use to_categorical, as we did in the lectures, to transform both *y_train* and *y_test* into one-hot encoded vectors of length *num_classes*:

In [34]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

## Model definition

Next, initialise a Keras *Sequential* model and add three layers to it:

    Layer: Add a *Dense* layer with in input_shape=(max_words,), 512 output units and "relu" activation.
    Layer: Add a *Dropout* layer with dropout rate of 50%.
    Layer: Add a *Dense* layer with num_classes output units and "softmax" activation.

In [35]:
model = Sequential()  # Instantiate sequential model
model.add(Dense(512,activation='relu',input_shape=(max_words,))) # Add first layer. Make sure to specify input shape
model.add(Dropout(0.5)) # Add second layer
model.add(Dense(num_classes,activation='softmax')) # Add third layer

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 512)               512512    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 46)                23598     
Total params: 536,110
Trainable params: 536,110
Non-trainable params: 0
_________________________________________________________________


## Model compilation

As the next step, we need to compile our Keras model with a training configuration. Compile your model with "categorical_crossentropy" as loss function, "adam" as optimizer and specify "accuracy" as evaluation metric. NOTE: In case you get an error regarding h5py, just restart the kernel and start from scratch

In [36]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',
             metrics = ['accuracy'])

In [37]:
#some learners constantly reported 502 errors in Watson Studio. 
#This is due to the limited resources in the free tier and the heavy resource consumption of Keras.
#This is a workaround to limit resource consumption

from keras import backend as K

K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))


## Model training and evaluation


In [38]:
batch_size = 32
model.fit(x_train,y_train,
         batch_size=batch_size,
         epochs=5,
         validation_data=(x_test,y_test))

score = model.evaluate(x_test,y_test,verbose=0)

Train on 8982 samples, validate on 2246 samples
Epoch 1/5
8982/8982 [==============================] - 5s 562us/step - loss: 1.3969 - acc: 0.6885 - val_loss: 0.9784 - val_acc: 0.7765
Epoch 2/5
8982/8982 [==============================] - 4s 463us/step - loss: 0.7714 - acc: 0.8192 - val_loss: 0.8214 - val_acc: 0.8090
Epoch 3/5
8982/8982 [==============================] - 4s 471us/step - loss: 0.5568 - acc: 0.8652 - val_loss: 0.8174 - val_acc: 0.8099
Epoch 4/5
8982/8982 [==============================] - 4s 478us/step - loss: 0.4190 - acc: 0.8926 - val_loss: 0.8370 - val_acc: 0.8010
Epoch 5/5
8982/8982 [==============================] - 4s 485us/step - loss: 0.3466 - acc: 0.9107 - val_loss: 0.8556 - val_acc: 0.7988


In [39]:
# Save Model
model.save("text_classify.h5")

In [45]:
from keras.models import load_model

# load Model
model = load_model("text_classify.h5")

In [46]:
score = model.evaluate(x_test,y_test,verbose=0)
score[1]

0.798753339269813

## 2) Train with LSTM RNN


In [17]:
#Import Library
import tensorflow as tf
import numpy as np

#Keras Modules
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM

#Data Visualization depedencies
import scikitplot as skplt
from matplotlib import pyplot as plt
%matplotlib inline

#Preprocessing
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

## Import Dataset

In [18]:
from keras.datasets import reuters

max_words = 1000
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2,
                                                         seed=seed)
num_classes = np.max(y_train) + 1  # 46 topics



## Preprocessing

## Tokenization + Categorical Encoding

In [19]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

## Train the Model

In [27]:
# Create Model

model = Sequential()
#Embedded First Layer
model.add(Embedding(max_words,128))
#LSTM RNN Layer
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2))
#Dense Output Layer : Review - Like or Dislike (1,0)
model.add(Dense(num_classes,activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 128)         128000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_6 (Dense)              (None, 46)                5934      
Total params: 265,518
Trainable params: 265,518
Non-trainable params: 0
_________________________________________________________________


In [28]:
#Compile the Model
model.compile(loss='binary_crossentropy',optimizer='adam',
              metrics=['accuracy'])

In [29]:
#Train the Model
model.fit(x_train,y_train,
         batch_size=32,
         epochs=1,
         validation_data=(x_test,y_test))

Train on 8982 samples, validate on 2246 samples
Epoch 1/1
8982/8982 [==============================] - 819s 91ms/step - loss: 0.1016 - acc: 0.9727 - val_loss: 0.0719 - val_acc: 0.9783


In [30]:
#Evaluate the model
score = model.evaluate(x_test,y_test,verbose=0)
score[1]

0.9782608745890012